### Merging all paquet files at start of job...


In [28]:
import pandas as pd
import numpy as np
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType

spark = SparkSession.builder.master("local[*]").appName("Learning_Spark").getOrCreate()

directory = "landing/persistent/idealista"

file_names = []  # List which will store all of the full filepaths.
# Walk the tree.
for root, directories, files in os.walk(directory):
    for filename in files:
        if filename[-7:] == 'parquet':
            file_names.append(root+'/'+filename)
            #path_df = spark.read.parquet(root+'/'+filename)
            #idealista_df = idealista_df.union(path_df)
            
# fill spark dataframe
idealista_df = spark.read.parquet(file_names[0])

for file in file_names[1:]:
    idealista_df = idealista_df.union(file)


128

### Misc testing...

In [29]:
file_names

['landing/persistent/idealista/2021_02_22_idealista/part-00000-10c3d7de-11b8-44a9-907f-6f0a7d7051d0-c000.snappy.parquet',
 'landing/persistent/idealista/2020_03_16_idealista/part-00000-88f16d66-e7f6-4fa0-9ab8-bac88dc5adfd-c000.snappy.parquet',
 'landing/persistent/idealista/2020_01_13_idealista/part-00000-0b5efacb-48eb-472a-9b87-a863fe92e503-c000.snappy.parquet',
 'landing/persistent/idealista/2020_02_10_idealista/part-00000-5dcf60c1-1a7f-41f2-967e-5cead88ecf36-c000.snappy.parquet',
 'landing/persistent/idealista/2020_06_13_idealista/part-00000-c6a1025d-3527-418f-a90d-eb45a03f51ed-c000.snappy.parquet',
 'landing/persistent/idealista/2020_04_16_idealista/part-00000-5271a859-1abd-4195-8c2c-a6cf2a4ab961-c000.snappy.parquet',
 'landing/persistent/idealista/2020_07_15_idealista/part-00000-a946f2f4-f32a-405c-aebe-5a78c8a06979-c000.snappy.parquet',
 'landing/persistent/idealista/2020_10_25_idealista/part-00000-3653c01d-41ce-4d89-bb3b-a360abeb2106-c000.snappy.parquet',
 'landing/persistent/ide

In [32]:
idealista_df.printSchema()

root
 |-- address: string (nullable = true)
 |-- bathrooms: long (nullable = true)
 |-- country: string (nullable = true)
 |-- detailedType: struct (nullable = true)
 |    |-- subTypology: string (nullable = true)
 |    |-- typology: string (nullable = true)
 |-- distance: string (nullable = true)
 |-- district: string (nullable = true)
 |-- exterior: boolean (nullable = true)
 |-- externalReference: string (nullable = true)
 |-- floor: string (nullable = true)
 |-- has360: boolean (nullable = true)
 |-- has3DTour: boolean (nullable = true)
 |-- hasLift: boolean (nullable = true)
 |-- hasPlan: boolean (nullable = true)
 |-- hasStaging: boolean (nullable = true)
 |-- hasVideo: boolean (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- municipality: string (nullable = true)
 |-- neighborhood: string (nullable = true)
 |-- newDevelopment: boolean (nullable = true)
 |-- numPhotos: long (nullable = true)
 |-- operation: string (nullable =